# Library

In [70]:
import os
import pandas as pd
import re
import numpy as np
import warnings

In [71]:
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

pd.options.display.float_format = '{:,.4f}'.format

# Path Setting

In [2]:
# Use the current working directory instead
base_path = os.getcwd()
data_source_dir = os.path.abspath(os.path.join(base_path, "..", "..", "A. Data Source", "A.1. SEKI (Bank Indonesia)"))
data_result_dir = os.path.abspath(os.path.join(base_path, "..", "..", "C. Processed Data", "C.1. SEKI (Bank Indonesia)"))

# Additional Function

In [12]:
def read_sheet_7_1(file_path):
    try:
        # Get all sheet names
        sheet_names = pd.ExcelFile(file_path).sheet_names

        # Select the appropriate sheet
        target_sheet = None
        if '7.1' in sheet_names:
            target_sheet = '7.1'
        else :
            target_sheet = sheet_names[-1]

        if target_sheet:
            df = pd.read_excel(file_path, sheet_name=target_sheet)
            return df
        else:
            print(f"No sheet '7.1'found in {file_path}")
            return None

    except Exception as e:
        print(f"Failed to read {file_path}: {e}")
        return None

In [85]:
def fill_first_row_pattern(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fill null values in the first row of a DataFrame based on Q1, Q4, Q3 rules.
    Starting from column index 1. Pattern check is now 'contains' instead of '=='.
    Skips filling if start == end - 1 (already only one cell).
    """
    df = df.copy()
    first_row = df.iloc[0].copy()

    col_index = 1  # Start from column index 1
    while col_index < len(df.columns):
        if pd.notna(first_row[col_index]):
            value = first_row[col_index]

            if pd.notna(df.iloc[1, col_index]):
                pattern = str(df.iloc[1, col_index]).strip().upper()

                if "Q1" in pattern:
                    # print(f"Found Q1 pattern : {str(value)}")
                    start = col_index
                    end = min(col_index + 5, len(df.columns))  # Next 4 columns
                    if start != end - 1:  # Skip if only 1 column
                        for i in range(start, end):
                            first_row[i] = str(value)
                    col_index = end
                    continue

                elif "Q4" in pattern:
                    # print(f"Found Q4 pattern : {str(value)}")
                    start = max(col_index - 3, 0)  # 3 before
                    end = min(col_index + 2, len(df.columns))  # 1 after
                    if start != end - 1:
                        for i in range(start, end):
                            first_row[i] = str(value)
                    col_index = end
                    continue

                elif "Q3" in pattern:
                    # print(f"Found Q3 pattern : {str(value)}")
                    start = max(col_index - 2, 0)  # 2 before
                    end = col_index + 1
                    if start != end - 1:
                        for i in range(start, end):
                            first_row[i] = str(value)
                    col_index = end
                    continue

        col_index += 1

    df.iloc[0] = first_row
    return df

In [88]:
def normalize_text(text):
    """Convert text to lowercase and remove all spaces."""
    return str(text).lower().replace(" ", "")

def merge_rows(df, merge_list, col_index=1):
    # Normalize the merge_list
    normalized_list = [normalize_text(item) for item in merge_list]

    rows_to_drop = []
    for i in range(len(df) - 1):
        current_val = normalize_text(df.iloc[i, col_index])

        if current_val in normalized_list:
            # Merge current row with the next row (column by column)
            for col in df.columns:
                val1 = df.at[i, col]
                val2 = df.at[i + 1, col]

                # Convert nulls to empty string, others to string
                str1 = "" if pd.isna(val1) else str(val1)
                str2 = "" if pd.isna(val2) else str(val2)

                # Merge with a space only if both are non-empty
                if str1 and str2:
                    merged = str1 + " " + str2
                else:
                    merged = str1 + str2  # One of them is empty

                df.at[i, col] = merged

            rows_to_drop.append(i + 1)

    df = df.drop(rows_to_drop).reset_index(drop=True)
    return df

In [94]:
def rename_columns_from_first_row(df: pd.DataFrame) -> pd.DataFrame:
    # Step 2: Get the first row as new names (for columns index 3 onward)
    new_names = df.iloc[0].tolist()

    # Step 3: Combine fixed names (first 3) + new names from first row
    updated_cols = new_names

    # Step 4: Assign new column names
    df.columns = updated_cols

    # Step 5: Drop the first row
    df = df.iloc[1:].reset_index(drop=True)

    return df

In [105]:
def transform_df_long(df:pd.DataFrame):
    # Identify year-based columns
    year_cols = [col for col in df.columns if re.match(r'^\d{4}', str(col))]
    other_cols = [col for col in df.columns if col not in year_cols]

    # Melt: POSISI and KOMPONEN both as rows
    df_long = df.melt(
        id_vars= other_cols,
        value_vars=year_cols,
        var_name='POSISI',
        value_name='VALUE'
    )

    return df_long

In [110]:
def end_to_end_df_processing(df:pd.DataFrame) -> pd.DataFrame :
    merge_key_list = [
        "LAPANGAN USAHA"
    ]
    
    df = df.iloc[3:].reset_index(drop=True)
    df = df.drop(df.columns[[0, 1]], axis=1)
    df = df.drop(df.columns[-4:], axis=1)
    df = df.dropna(axis=1, how='all')
    df = fill_first_row_pattern(df)
    df = merge_rows(df, merge_key_list, 0)
    # Add feature index for ordering based on feature
    df = rename_columns_from_first_row(df)
    df = df.reset_index()
    df = df.rename(columns={'index': 'feature_index'})
    df = df.iloc[:-2]
    df = transform_df_long(df)
    
    return(df)

In [109]:
def combine_two_dfs(df_list : list) -> pd.DataFrame :
    key_cols = ['LAPANGAN USAHA', 'POSISI']

    combined = pd.concat(df_list, ignore_index=True)
    combined = combined.drop_duplicates(subset=key_cols, keep='last')
    return(combined.reset_index(drop=True))

# Main Code

In [ ]:
file_to_be_extracted = "TABEL7_1.xls"
data_dict = {}

# Loop through all child folders
for folder in os.listdir(data_source_dir):
    folder_path = os.path.join(data_source_dir, folder)
    
    # Only process if it's a directory
    if os.path.isdir(folder_path):
        file_path = os.path.join(folder_path, file_to_be_extracted)
        try:
            df = read_sheet_7_1(file_path)
            key = os.path.splitext(folder + "_" + file_to_be_extracted)[0]  # Get filename without extension
            data_dict[key] = df
        except Exception as e:
            print(f"Failed to read {file_path}: {e}")

In [17]:
data_dict_backup = data_dict.copy()

In [111]:
data_dict = data_dict_backup.copy()
combined_all_df = pd.DataFrame()
key_list_ordered = [
    'SEKI_JUNI_2022_TABEL7_1', 'SEKI_DESEMBER_2022_TABEL7_1', 
    'SEKI_JUNI_2023_TABEL7_1', 'SEKI_DESEMBER_2023_TABEL7_1', 
    'SEKI_JUNI_2024_TABEL7_1', 'SEKI_DESEMBER_2024_TABEL7_1',
    'SEKI_JUNI_2025_TABEL7_1'
]

for key in key_list_ordered :
    df = data_dict[key]
    new_df = end_to_end_df_processing(df)
    data_dict[key] = new_df
    combined_all_df = combine_two_dfs([combined_all_df, new_df])

combined_all_df = combined_all_df.sort_values(by=['POSISI', 'feature_index'], ascending=[True, True]).reset_index(drop=True)

In [113]:
new_column_order = [
    'POSISI', 'POSISI_TYPE', 'feature_index', 'LAPANGAN USAHA', 'SATUAN/ UNIT', 
    'VALUE'
]

combined_all_df['POSISI_TYPE'] = np.where(combined_all_df['POSISI'].str.endswith('**'), 'Angka Sangat Sementara',
                     np.where(combined_all_df['POSISI'].str.endswith(' *'), 'Angka Sementara',
                              'Angka Tetap'))

combined_all_df['SATUAN/ UNIT'] = "(Miliar Rp)"
combined_all_df = combined_all_df.sort_values(by=['POSISI', 'feature_index'], ascending=[True, True]).reset_index(drop=True)

combined_all_df = combined_all_df[new_column_order]

In [114]:
combined_all_df

,POSISI,POSISI_TYPE,feature_index,LAPANGAN USAHA,SATUAN/ UNIT,VALUE
0,2010,Angka Tetap,0,"PERTANIAN, KEHUTANAN & PERIKANAN",(Miliar Rp),"956,119.7000"
1,2010,Angka Tetap,1,"Pertanian, Peternakan, Perburuan dan Jasa Pert...",(Miliar Rp),"754,434.4000"
2,2010,Angka Tetap,2,Tanaman Pangan,(Miliar Rp),"253,326.6000"
3,2010,Angka Tetap,3,Tanaman Hortikultura,(Miliar Rp),"110,395.3000"
4,2010,Angka Tetap,4,Tanaman Perkebunan,(Miliar Rp),"268,207.3000"
...,...,...,...,...,...,...
5947,2025 Q1**,Angka Sangat Sementara,59,JASA KESEHATAN DAN KEGIATAN LAINNYA,(Miliar Rp),"69,914.1000"
5948,2025 Q1**,Angka Sangat Sementara,60,JASA LAINNYA,(Miliar Rp),119564
5949,2025 Q1**,Angka Sangat Sementara,61,NILAI TAMBAH BRUTO ATAS HARGA DASAR,(Miliar Rp),"5,443,989.1000"
5950,2025 Q1**,Angka Sangat Sementara,62,PAJAK DIKURANG SUBSIDI ATAS PRODUK,(Miliar Rp),"221,941.1000"


# Export Result

In [115]:
result_file_name = "SEKI_7_1_result.xlsx"

result_file_path = os.path.join(data_result_dir, result_file_name)

In [116]:
combined_all_df.to_excel(result_file_path, index=False)

In [117]:
result_df = pd.read_excel(result_file_path)

In [122]:
result_df = result_df.rename(
    columns={
        "feature_index": "FEATURE_INDEX",
        "LAPANGAN_USAHA":"SEKTOR_EKONOMI",
        "SATUAN/ UNIT":"SATUAN_UNIT"
    }
)

# Replace only exact "-"
result_df["VALUE"] = result_df["VALUE"].replace("-", np.nan)

In [124]:
result_df.to_excel(result_file_path, index=False)